In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import os
import pickle
import math

from classification.datasets import Dataset
from classification.utils.audio_student import AudioUtil, Feature_vector_DS
from classification.utils.plots import (
    plot_decision_boundaries,
    plot_specgram,
    show_confusion_matrix,
)
from classification.utils.utils import accuracy

In [ ]:
# Load the dataset
dataset = Dataset()
classnames = dataset.list_classes() #['chainsaw', 'fire', 'fireworks', 'gun']








In [ ]:
# Hyperparameters other than those of the model
Nft_list = 512 # number of samples in the Fourier transform
nmel_list = 20 # number of mel bands
pca_list = [math.ceil(1020/Nft_list[0]) * nmel_list[0] / 2] # number of components in PCA
threshold = 0.05 # threshold for the power of the signal